In [4]:
#The first step is to extract/create data from the music (.wav) files using librosa
import librosa
import os
import pandas as pd
import pickle
import numpy as np

In [5]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
    
#Open each .wav file, use the audio information to generate useful metrics for our data
#Metrics including:  root mean square energy, spectral_centroid
#spectral_bandwidth, spectral_rolloff, zero_crossing_rate and mfcc (Mel-Frequency Cepstral Coefficients)
dataset = []
for g in genres:
    for song in os.scandir(f'./genres/{g}'):
        genre_label = g
        track, sr = librosa.load(f'./genres/{g}/{song.name}', duration = 30)
        rms = librosa.feature.rms(y = track).reshape(-1,)
        zcr = librosa.feature.zero_crossing_rate(track).reshape(-1,)
        spec_bw = librosa.feature.spectral_bandwidth(y=track, sr=sr).reshape(-1,)
        spec_cent = librosa.feature.spectral_centroid(y = track, sr=sr).reshape(-1,)
        rolloff = librosa.feature.spectral_rolloff(y=track, sr=sr).reshape(-1,)
        mfcc = librosa.feature.mfcc(y=track, sr=sr)
        
        items_to_append = [rms, zcr, spec_bw, spec_cent, rolloff]
        for c in mfcc:
            items_to_append.append(c)
            
        items_to_append.append(genre_label)
        
        dataset.append(items_to_append)

In [6]:
pickle.dump(dataset, open( "songData.p", "wb" ) )

In [7]:
data = pickle.load( open( "songData.p", "rb" ) )
raw_data = data.copy()

In [8]:
features = []
label = []
for i in range(len(raw_data)):
    features.append(raw_data[i][:-1])
    label.append(raw_data[i][-1])

In [9]:
feature_df = pd.DataFrame(data = features)
label_df = pd.DataFrame(data = label)
feature_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,"[0.026987005, 0.03934191, 0.0497628, 0.0639432...","[-211.48465, -208.94481, -193.90889, -174.6187...","[99.022964, 101.24678, 102.24396, 107.01251, 1...","[-10.634697, -9.346699, 1.915434, -2.8550763, ...","[38.72212, 44.962704, 53.66914, 55.168877, 51....","[2.8948734, 3.8570495, 3.0936985, 9.057784, 2....","[20.152992, 21.218014, 27.13928, 29.364662, 36...","[-19.852455, -15.464728, -14.800985, -17.80081...","[2.7951534, 1.7642274, 9.898521, 9.002836, 10....","[-7.818969, -9.43764, -6.462226, -1.5302008, 3...",...,"[-2.1509945, -3.7085586, -9.185015, -15.804844...","[-2.5450845, 0.64133763, -2.1272182, -3.417891...","[-2.3836384, -1.6919363, -3.844904, -2.4906926...","[15.9462185, 12.448543, 9.030369, 9.808171, 13...","[4.60089, 0.7752062, 2.9174325, -2.4179955, -1...","[-6.3438263, -8.134716, -6.261236, -7.4471, -5...","[0.30703503, -0.10146806, -3.9532547, -0.37419...","[8.944412, 6.8827453, 3.6500254, -0.31883085, ...","[3.074852, 4.5667257, 4.7889977, -2.4243538, -...","[5.0415277, 2.8966317, 0.94024646, 1.0172188, ..."
1,"[0.06771811, 0.107166275, 0.14316952, 0.152401...","[-312.9025, -223.67754, -156.06529, -148.91068...","[164.21045, 144.91583, 133.97658, 125.30925, 1...","[6.496667, 12.937246, 11.931242, 17.2429, 23.3...","[22.498564, 43.057713, 48.249382, 55.611862, 5...","[16.248466, -0.6615118, -4.4315314, -2.8002713...","[7.0293975, 33.964508, 44.50891, 41.24395, 30....","[9.781713, 4.040926, 1.0407987, -7.1708093, -1...","[-0.5247682, 13.222952, 21.73407, 24.12668, 24...","[-12.539514, -4.654312, -1.8885226, -0.7466556...",...,"[1.7854517, 1.2485096, -1.9579388, -4.6947145,...","[-8.288733, 3.4144387, 8.513353, 12.536104, 16...","[-4.8438225, -4.6451693, -1.9787173, 1.7404395...","[0.25834954, 2.7501483, 7.5640173, 7.127599, 5...","[-5.6865973, -7.650701, -10.392296, -9.318182,...","[-1.11748, -1.5572802, 4.604141, 7.8723707, 4....","[6.026271, 0.9509426, -4.3049135, -5.2468796, ...","[4.5413375, 1.5771381, 4.937603, 6.2523394, 0....","[-1.8412286, -3.5716817, -0.31534874, 1.630790...","[-2.8207376, 0.20880526, 1.7589519, 0.97758806..."
2,"[0.17388117, 0.17076328, 0.15829462, 0.1419233...","[-60.05452, -71.44897, -96.38281, -108.587616,...","[147.24527, 149.00153, 141.7526, 140.55637, 14...","[-53.251102, -44.86586, -32.56984, -30.859358,...","[9.966818, 15.332075, 24.36048, 25.233881, 24....","[-4.3276243, -3.7276168, -5.071242, -6.7992706...","[37.57383, 36.255684, 33.475765, 33.772877, 38...","[-2.4147024, -4.8258467, -6.351204, -13.37511,...","[11.080793, 11.4219265, 9.938736, 6.860955, 5....","[-15.251921, -15.562403, -11.569693, -14.76624...",...,"[-21.794657, -17.329823, -8.680307, -6.318611,...","[5.590602, 7.1561055, 10.241064, 11.845917, 10...","[-11.437779, -16.062212, -16.299515, -9.13135,...","[0.91903895, -2.9200811, -13.372895, -12.83487...","[-2.540476, -7.2031517, -13.193421, -6.4380655...","[4.7461405, 3.1809878, 4.9475975, 7.9952364, 9...","[-9.244363, -11.065447, -12.8444805, -12.40548...","[-4.347864, -3.973063, -3.4759495, -4.3304057,...","[-4.427139, -6.135474, -6.4472322, 2.8229146, ...","[-2.8976831, -4.329397, -8.008919, -5.815725, ..."
3,"[0.15084551, 0.15889509, 0.16152988, 0.1566745...","[-187.64453, -196.83621, -213.41223, -220.4217...","[183.05563, 180.47028, 178.12717, 181.27698, 1...","[-4.1095085, -2.1412551, -1.8208245, -6.030243...","[22.43245, 25.776375, 27.863049, 25.056187, 24...","[8.586718, 7.5933743, 8.384164, 11.577272, 16....","[10.318501, 11.128399, 7.5562687, 3.1053128, 6...","[1.5045615, 4.9668045, 8.759226, 7.0657387, 5....","[16.393448, 16.561039, 17.82682, 18.210474, 17...","[15.119045, 14.121504, 14.153143, 19.301462, 1...",...,"[3.2138944, 2.4555385, 3.2412612, 8.939145, 12...","[9.056037, 9.871952, 8.425485, 5.493551, 4.754...","[3.9844403, 4.6813116, 3.4601312, 0.34149444, ...","[8.357418, 4.81915, 2.0221238, 1.1993222, 1.81...","[6.760809, 5.661574, 2.464192, 1.1221669, -1.0...","[3.296612, 1.3385847

In [10]:
feature_df = feature_df.drop(432)
label_df = label_df.drop(432)

In [11]:
# standardizing each feature

# calculating average for each feature
# since each sample has the same number of values, we can just average the averages.
feature_average = []
for c in range(len(feature_df.columns)):
    sub_average = []
    for sample in feature_df.iloc[:][c].values:
        sub_average.append(np.average(sample))
        
    feature_average.append(np.average(sub_average))

# calculating standard deviation
n_population = [] # total number of measurements per feature(accounting for each one's length in time)
feature_stdev = [] # the end goal to calculate for each feature so that I can scale the entire feature
sample_stdev_num = [] # the numerator of the stdev for each sample
sample_stdev_num_sum = [] #the sum of each numerator so that the stdev is calculated for the entire population

for c in range(len(feature_df.columns)):
    n_population.append(len(feature_df)*len(feature_df.iloc[0][c]))
    sample_stdev_num = []
    sample_stdev_num_sum = []
    for sample in feature_df.iloc[:][c].values:
        sample_stdev_num.append(np.square(sample-feature_average[c]))
        
    sample_stdev_num_sum = np.sum(sample_stdev_num)
    feature_stdev.append(np.sqrt(sample_stdev_num_sum/n_population[c]))
     

# standardizing all features now that we know stdev and the mean value
for c in range(len(feature_df.columns)):
    for i in range(len(feature_df.iloc[:][c])):
        feature_df.iloc[i][c] = (feature_df.iloc[i][c]-feature_average[c])/feature_stdev[c]

In [12]:
len(feature_df.iloc[0][10])

1292

In [13]:
label_df = pd.get_dummies(label_df)
feature_df.columns =("RMS", "ZCR", "Spec_Bandwidth", "Spec_Centroid", "Spectral_Rolloff", "MFCC1"
                            ,"MFCC2", "MFCC3", "MFCC4", "MFCC5", "MFCC6", "MFCC7"
                            , "MFCC8", "MFCC9", "MFCC10", "MFCC11", "MFCC12", "MFCC13"
                            , "MFCC14", "MFCC15", "MFCC16", "MFCC17", "MFCC18", "MFCC19", "MFCC20")

label_df.columns =("Blues", "Classical", "Country", "Disco", "Hiphop", "Jazz", "Metal", "Pop", "Reggae", "Rock")

In [14]:
pickle.dump( feature_df, open( "preprocessedFeat.p", "wb" ) )
pickle.dump( label_df, open( "preprocessedLabel.p", "wb" ) )